In [ ]:
# context_template에 3가지 정보를 넣어서 evaluation으로 쓴다는게 특이점. 알아는 두기
def CRAG_evaluation(question, ground_truth, prediction):
  context_template = f"""Question: {question}
  List of Ground Truth answers: {ground_truth}
  Model Prediction: {prediction}
  """

  evaluation_result = generate_answer(user_prompt=context_template, system_prompt=INSTRUCTIONS)

  eval_res = parse_response(evaluation_result)

  return eval_res

In [ ]:
# 맨 밑에 CRAG_score 식 정도는 외워도 ? 
# CRAG_score = n_correct_exact + 0.5*n_correct - n_hallucinate

for data in tqdm(dataset):
  if data['interaction_id'] not in finance_test_dataset_ids:
    continue

  question = data['query']
  ground_truth_lowercase = str(data['answer']).strip().lower()
  web_search_results = data['search_results']

  prediction_lowercase = rag.inference(question, web_search_results, 5)['answer'].lower()

  if prediction_lowercase == ground_truth_lowercase:
      n_correct_exact += 1
      continue
  elif "i don't know" in prediction_lowercase:
      n_miss += 1
      continue

  acceptable = CRAG_evaluation(question, ground_truth_lowercase, prediction_lowercase)

  if acceptable == 1:
    n_correct += 1

n_hallucinate = (len(finance_test_dataset_ids) - n_correct_exact - n_correct - n_miss)

CRAG_score = n_correct_exact + 0.5*n_correct - n_hallucinate



In [ ]:
# MCP랑 llama중에 하나 나온다고 하니 꼭 외우기
external_mcp_server = "http://x.x.x.x:port/sse" #change to correct uri

mcp_client = BasicMCPClient(external_mcp_server)
mcp_tool = McpToolSpec(client=mcp_client)

# await가 들어가는 것 잘 외우기
tools = await mcp_tool.to_tool_list_async()
print("\n=== Available Tools ===\n")
for tool in tools:
    print(f"🔧 Name       : {tool.metadata.name}")
    print(f"   Description: {tool.metadata.description}\n")

resources = await mcp_tool.fetch_resources()
print("\n=== Available Resources ===\n")
for resource in resources:
    print(f"📦 URI        : {resource.uri}")
    print(f"   Name       : {resource.name}")
    print(f"   Description: {resource.description}")
    print(f"   MIME Type  : {resource.mimeType}\n")

In [ ]:
# 기본적인 문법이랑 async 랑 await가 어디에 붙는지 알아둬야 할거 같은?
# await는 비동기(async) 함수가 끝날때까지 기다린 후 결과를 받아오는 것.
SYSTEM_PROMPT = """\
You are an AI assistant for Tool Calling.

Before you help a user, you need to work with tools to interact with Our Knowledge Graph
"""
```

```python
from llama_index.llms.openai import OpenAI

class KGQueryEngineWithMCP:
    def __init__(self, mcp_tool_spec: McpToolSpec, model: str, llm = None):
        self.llm = llm or OpenAI(model=model, temperature=0)
        self.mcp_tool_spec = mcp_tool_spec
        self.agent: Optional[FunctionAgent] = None
        self.agent_context: Optional[Context] = None

    async def init_agent(self):
        tools = await self.mcp_tool_spec.to_tool_list_async()
            # self.mcp_tool_spec는 MCP에서 정의된 툴 스펙(tool specification) 객체
            # to_tool_list_async() 메서드는 이 스펙을 실제 툴 리스트(list of tools)로 변환하는 비동기 함수
            # await 키워드는 비동기 함수의 결과를 기다린다는 의미입니다.
            # 즉, to_tool_list_async()가 완료될 때까지 기다린 후, 반환된 결과(툴 리스트)를 tools 변수에 저장
        self.agent = FunctionAgent(
            name="Agent",
            description="An agent that can work with Our Knowledge Graph api.",
            tools=tools,
            llm=self.llm,
            system_prompt=SYSTEM_PROMPT,
        )
        self.agent_context = Context(self.agent)

    async def query(self, question: str, verbose: bool = False) -> str:
        if self.agent is None or self.agent_context is None:
            raise RuntimeError("Agent not initialized. Call `await init_agent()` first.")

        handler = self.agent.run(question, ctx=self.agent_context)
        async for event in handler.stream_events():
            if verbose and type(event) == ToolCall:
                print(f"Calling tool {event.tool_name} with kwargs {event.tool_kwargs}")
            elif verbose and type(event) == ToolCallResult:
                print(f"Tool {event.tool_name} returned {event.tool_output}")

        response = await handler
        return str(response)

kg_engine = KGQueryEngineWithMCP(mcp_tool, model='gpt-3.5-turbo')
await kg_engine.init_agent() # 위에 async로 선언해놓은 init_agent 함수를 호출할 때 await 붙이냐는 문제가 나올수 ???
response = await kg_engine.query(
    question,
    verbose=True,
)

In [ ]:
""" RAGWithKG와 다른점
    1) kg_query_engine 대신 mcp_application으로 선언
    2) retrieve와 inference가 async로 선언 됨 <- retrieve의 init_agent()와 query를 await로 받아와야 해서 함수 자체는 async 
    3) 실제 inference를 호출할때도 await로 호출해야 함.
"""
class RAGwithMCP:
    def __init__(self, server=None):
        self.mcp_application = KGQueryEngineWithMCP(server=server)
        self.reader = Reader()

    async def retrieve(self, query: str, query_time: str, search_results: list, topk: int):
        await self.mcp_application.init_agent()

        full_query = f"""Query: {query} 
                         Query time: {query_time}"""

        mcp_result = await self.mcp_application.query(full_query, verbose=False)
        return mcp_result

    def generate_response(self, query: str, query_time: str, retrieved_results: str):
        answer = self.reader.generate_response(query, query_time, retrieved_results)
        return answer

    async def inference(self, query: str, search_results: list, query_time: str, topk: int):
        retrieved_results = await self.retrieve(query, query_time, search_results, topk)
        answer = self.generate_response(query, query_time, retrieved_results)
        return {
            "retrieved_results": retrieved_results,
            "answer": answer
        }

rag_mcp = RAGwithMCP(server=external_mcp_server)
result = await rag_mcp.inference(
    query=question,
    search_results=[],
    query_time=query_time,
    topk=5
)
